https://www.tensorflow.org/guide/estimator?hl=es-419

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from opacus import PrivacyEngine

# Set random seed for reproducibility
torch.manual_seed(42)

# Hyperparameters
train_size = 900
test_size = 1000
l2_lambda = 1e-4  # Weight decay
noise_multiplier = 1.0  # Noise multiplier for DP
max_grad_norm = 1.0  # Clip the gradients

# Data generation (parity check on first 3 bits, 30-bit input)
unique_binary_strings = set()
while len(unique_binary_strings) < train_size + test_size:
    binary_string = tuple(np.random.randint(2, size=30))
    unique_binary_strings.add(binary_string)

inputs = np.array(list(unique_binary_strings), dtype=np.float32)
outputs = np.sum(inputs[:, :3], axis=-1) % 2  # Parity check on first 3 bits

# Convert data to PyTorch tensors
inputs = torch.tensor(inputs, dtype=torch.float32)
outputs = torch.tensor(outputs, dtype=torch.float32).view(-1, 1)

# Split into train and evaluation sets
indices = torch.randperm(len(inputs))
split_idx = train_size
X_train, y_train = inputs[indices[:split_idx]], outputs[indices[:split_idx]]
X_eval, y_eval = inputs[indices[split_idx:]], outputs[indices[split_idx:]]

# Define the MLP model with L2 regularization
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(31, 30)  # Input dim + 1 for bias
        self.fc2 = nn.Linear(30, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        return torch.sigmoid(self.fc2(x))

# Instantiate the model, define loss and optimizer
model = MLP()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), weight_decay=l2_lambda)

# Attach Privacy Engine
privacy_engine = PrivacyEngine()
model, optimizer, data_loader = privacy_engine.make_private(
    module=model,
    optimizer=optimizer,
    data_loader=data_loader,
    noise_multiplier=1.1,
    max_grad_norm=1.0,
)

# Training loop
num_epochs = 500
train_accuracies = []
test_accuracies = []

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    outputs_train = model(X_train)
    loss = criterion(outputs_train, y_train)

    # Backward pass
    loss.backward()
    optimizer.step()
    
    # Get train accuracy
    with torch.no_grad():
        train_accuracy = ((outputs_train > 0.5).float() == y_train).float().mean().item()
        train_accuracies.append(train_accuracy)

    # Evaluate on the test set
    model.eval()
    outputs_eval = model(X_eval)
    test_accuracy = ((outputs_eval > 0.5).float() == y_eval).float().mean().item()
    test_accuracies.append(test_accuracy)

    # Step the privacy engine
    privacy_engine.step()

    if (epoch + 1) % 50 == 0:
        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}, Train Acc: {train_accuracy:.4f}, Test Acc: {test_accuracy:.4f}")

# Evaluation
print(f"Final Test Accuracy: {test_accuracy * 100:.2f}%")

# Plot training and test accuracy over epochs
plt.plot


TypeError: PrivacyEngine.__init__() got an unexpected keyword argument 'sample_size'